In [1]:
import ee
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [2]:
# Import datasets
gaul = ee.FeatureCollection("FAO/GAUL/2015/level2")
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")

In [3]:
REGION_NAMES = ['El Jadida', 'Gabon', 'Isiolo', 'Khouribga', 'Kitui', 'Laikipia', 'Marsabit', 'Maynas', 'Para', 'Safi', 'Selangor', 'Sri Lanka']
REGION_TYPES = [2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 1, 0]
CHIRPS_SCALE = 250
MAX_PIXELS = 100000000

In [4]:
def process_chirps_image(image, county, scale, max_pixels):
    date = image.get('system:time_start')
    region_mask =  ee.Image.constant(1).clip(county)
    county_bounds = county.geometry().bounds()
    mask = region_mask
    masked_image = image.updateMask(mask)
    value = masked_image.select('precipitation').reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=county_bounds,
        scale=scale,
        maxPixels=max_pixels).get('precipitation')
    feature = ee.Feature(None, {'date': ee.Date(date).format('MMM dd, YYYY'),
                                'value': value})
    return feature


In [34]:
task_list = []
for region, region_type in zip(REGION_NAMES, REGION_TYPES):
    region_printable = region.lower().replace(' ', '')
    county = gaul.filter(ee.Filter.eq(f'ADM{region_type}_NAME', region))
    rainfall_data = chirps.map(lambda img: process_chirps_image(img, 
                                                                county, 
                                                                CHIRPS_SCALE, 
                                                                MAX_PIXELS))
    task = ee.batch.Export.table.toDrive(rainfall_data, f'chirps_{region_printable}', region_printable, selectors = 'date, value')
    task.start()
    task_list.append(task)

In [40]:
for task in task_list:
    status = task.status()
    description = status['description']
    state = status['state']
    if state != 'COMPLETED':
        print(state, description)